In [ ]:
# SECTION 2
# The only issue that are lacking some atm variables

In [2]:
import cfgrib
from ecmwf.opendata import Client
from ipyleaflet import Choropleth, Map, basemap_to_tiles, LayersControl
import ipywidgets as widgets
from IPython.display import display
from localtileserver import get_leaflet_tile_layer, TileClient
from Magics import macro as magics
from Magics.macro import *
import numpy as np
import pandas as pd
import rasterio
import requests
import rioxarray as rxr
import xarray as xr

In [7]:
#parameters to be imported from main widget
# dates_to_download = ['20230823', '20230824'] #only last five days are available
dates_to_download = [0, -1, -2] #today, yesterday, the day before yesterday

initial_lat_lon = (38, -90) #first cyclone position
final_lat_lon = (60, 90) #last cyclone position

In [4]:
# variable initialization
# variables to visualize
variables = ['msl', '2t', 'tp'] #, 10fgg15] #list of variables to visualize, 10fgg15 doesn't work right now

min_lat = initial_lat_lon[0]
min_lon = initial_lat_lon[1]

max_lat = final_lat_lon[0]
max_lon = final_lat_lon[1]

mean_lat = (max_lat - min_lat)/2
mean_lon = (max_lon - min_lon)/2

# Download and load data functions

**CAREFUL**: with this function we are downloading the 10 days ahead forecasts, so only the forecast 10 days from the the date provided

In [ ]:
def dwnl_atmdata(variables, dates):
    """
    Function which replicates the code written by Laura
    It downloads 10 day ahead forecasts from the dates specified in dates
    It need the dates from which the forecast starts
    """
    fnames = []
    for var in variables:            
        for d in dates:
            if var == "10fgg10":
                rqt = {
                    "date": str(d), #date start of the forecast
                    "time": 0,      #time start of the forecast, can be 0 or 12
                    "step": "216-240",    #step of the forecast: 10 days
                    "stream": "enfo",
                    "type": "ep",
                    "param": var,
                }
            else:
                rqt = {
                    "date": str(d), #date start of the forecast
                    "time": 0,      #time start of the forecast, can be 0 or 12
                    "step": 240,    #step of the forecast: 10 days
                    "stream": "oper",
                    "type": "fc",
                    "levtype": "sfc",
                    "param": var,
                }
            filename = f"data/atm/{var}_{rqt['date']}_time{rqt['time']}_step{rqt['step']}_{rqt['stream']}_{rqt['type']}.grib"
            client = Client(source = "ecmwf", beta = True)
            client.retrieve(
                request = rqt,
                target = filename
            )
            fnames.append(filename)
    return(fnames)

In [28]:
def dwnl_atmdata_today(variables):
    """
    Function which downloads 1, 2, 5 and 10 days ahead forecasts from today
    """
    fnames = []
    for var in variables:
        if var == "10fgg10": steps = ["0-24", "24-48", "96-120", "216-240"] else: steps = [24, 48, 120, 240]
        for s in steps:
            if var == "10fgg10":
                rqt = {
                    "date": 0,      #date start of the forecast: today
                    "time": 0,      #time start of the forecast, can be 0 or 12
                    "step": s,      #step of the forecast: 1, 2, 5, 10 days
                    "stream": "enfo",
                    "type": "ep",
                    "param": var,
                }
            else:
                rqt = {
                    "date": 0,      #date start of the forecast: today
                    "time": 0,      #time start of the forecast, can be 0 or 12
                    "step": s,      #step of the forecast: 1, 2, 5, 10 days
                    "stream": "oper",
                    "type": "fc",
                    "levtype": "sfc",
                    "param": var,
                }
            filename = f"data/atm/{var}_{rqt['date']}_time{rqt['time']}_step{rqt['step']}_{rqt['stream']}_{rqt['type']}.grib"
            client = Client(source = "ecmwf", beta = True)
            client.retrieve(
                request = rqt,
                target = filename
            )
            fnames.append(filename)
    return(fnames)

In [30]:
fnames = dwnl_atmdata_today(['msl', '2t', 'tp'])

20230826000000-24h-oper-fc.grib2:   0%|          | 0.00/229k [00:00<?, ?B/s]

20230826000000-48h-oper-fc.grib2:   0%|          | 0.00/231k [00:00<?, ?B/s]

20230826000000-120h-oper-fc.grib2:   0%|          | 0.00/232k [00:00<?, ?B/s]

20230826000000-240h-oper-fc.grib2:   0%|          | 0.00/234k [00:00<?, ?B/s]

20230826000000-24h-oper-fc.grib2:   0%|          | 0.00/280k [00:00<?, ?B/s]

...

20230826000000-240h-oper-fc.grib2:   0%|          | 0.00/279k [00:00<?, ?B/s]

20230826000000-24h-oper-fc.grib2:   0%|          | 0.00/306k [00:00<?, ?B/s]

20230826000000-48h-oper-fc.grib2:   0%|          | 0.00/347k [00:00<?, ?B/s]

20230826000000-120h-oper-fc.grib2:   0%|          | 0.00/405k [00:00<?, ?B/s]

20230826000000-240h-oper-fc.grib2:   0%|          | 0.00/399k [00:00<?, ?B/s]

In [6]:
# Load data
def gen_raster(var, d):
    """
    Generates .tiff raster files from .grib files downloaded from ECMWF's Open Data through dwnl_atmdata

    var: str
        Code of the variable
    d: str
        Date of the downloaded variable file

    Returns:
    tiffpath: str
        Path to the created .tiff file
    """
    filename = "data/atm/" + var + '_' + str(d) + 'time0_step240_oper_fc.grib' #add atm/ after data
    tiffpath = ".".join([filename.split(".")[0], "tiff"])
    f = xr.load_dataset(filename, engine = "cfgrib")
    f = f.rio.write_crs("epsg:4326")
    f.rio.to_raster(tiffpath)
    f.close()
    return(tiffpath)

def load_atmdata(varlst, dates):
    """
    varlst: str, list of str
        List containing strings of the variables to load
    dates: str, list of str
        List containing the dates for which data has been downloaded
    
    Returns:
    vardict: dict
        A dictionary of rasterio DatasetReader objects, each assigned to the corresponding variable
    """
    vardict = {}
    for i, var in enumerate(varlst):
        print(var, " conversion")
        lst = []
        for d in dates:
            lst.append(rasterio.open(gen_raster(var, d)))
        vardict[f"{var}"] = lst
    return(vardict)

def load_atmdata_today(varlst, fnames):
    """
    varlst: str, list of str
        List containing strings of the variables to load
    dates: str, list of str
        List containing the dates for which data has been downloaded
    
    Returns:
    vardict: dict
        A dictionary of rasterio DatasetReader objects, each assigned to the corresponding variable
    """
    vardict = {}
    for i, var in enumerate(varlst):
        print(var, " conversion")
        lst = []
        for d in dates:
            lst.append(rasterio.open(gen_raster(var, d)))
        vardict[f"{var}"] = lst
    return(vardict)

In [7]:
vardict = load_atmdata(all_variables, dates_to_download)

msl  conversion
2t  conversion
tp  conversion


In [ ]:
vardict = load_atmdata_today(all_variables, fnames)

In [16]:
#plot the data with ipyleaflet
def plot_atmdata(vardict, date, coord):
    """
    vardict: dict
        Dictionary containing the rasters uploaded trhough load_atmdata
    date: str, list of str
        Dates considered
    coord: list or tuple
        Coordinates of the map central point. Provide them as lat, lon
    """
    namedict = {
        "msl": "Mean sea level pressure [Pa]", #meansealevelpressure
        "2t": "2 metre temperature [K]",
        "tp": "Total Precipitation [m]",
        "10fgg15": "10 metre wind gust of at least 15 m/s [%]",
    }
    palettedict = {
        "msl": "cool", #meansealevelpressure
        "2t": "RdBu_r",
        "tp": "PuBu",
        "10fgg15": "winter",
    }
    m = Map(center=(coord[0], coord[1]), zoom = 3)
    for var in vardict.keys():
        r = [x for x in vardict[var] if date in x.name][0]
        print("Plotting ", namedict[var])
        client = TileClient(r)
        t = get_leaflet_tile_layer(client, name = namedict[var], opacity = 0.7, palette = palettedict[var])
        m.add_layer(t)
    m.add_control(LayersControl())
    m.layout.height = "700px"
    return(m)

In [17]:
coord = [mean_lat, mean_lon]
plot_atmdata(vardict, dates_to_download[0], coord)

Plotting  Mean sea level pressure [Pa]
Plotting  2 metre temperature [K]
Plotting  Total Precipitation [m]


Map(center=[11.0, 90.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

# WIDGETS

In [7]:
# LOAD AND DATA
all_variables = ['msl', '2t', 'tp']
gtp = []
gmsl = []
g2t = []
print('Loading data...')
for var in all_variables:
    print(var)
    for t in dates_to_download:
        filename = var + '_' + str(t) + 'time0_step240_oper_fc.grib'
        #plotname =  var + '_' + str(t) + 'time0_step240_oper_fc' 
        
        #output_grafic = output(output_formats = ['png'], output_name = plotname, 
        #                       output_name_first_page_number = "off" )

        #Loading Grib data
        gvar = mgrib(grib_input_file_name=filename)

        if var == 'tp':
            gtp.append(gvar)
        elif var == 'msl':
            gmsl.append(gvar)
        elif var == '2t':
            g2t.append(gvar)
print('Done!')

Loading data...
msl
2t
tp
Done!


In [9]:
# widgets selection lat lon
longitude = np.arange(-180,185,5)
longitude_slider = widgets.SelectionRangeSlider(
    options = longitude, index = (0,len(longitude)-1), description='Longitude',
    orientation='horizontal', disabled=False)
latitude = np.arange(-90,95,5)
latitude_slider = widgets.SelectionRangeSlider(
    options = latitude, index = (0,len(latitude)-1), description='Latitude',
    orientation='horizontal', disabled=False)

# widget select sea surface variable
widget_select_ssvar = widgets.Dropdown(description='Sea surface variable', options=('msl', '2t'))

# widget select atm variable
widget_select_atmvar = widgets.Dropdown(description='Atmospheric variable', options=('tp',))
                        # la coma del final és important; sinó agafa cada lletra per separat!!
# widget select timestep
vec_timesteps = list(range(0, len(dates_to_download))) 
widget_sel_timestep = widgets.SelectionSlider(options=vec_timesteps, description='Timestep')

# FUNCTION TO PLOT

In [45]:
def update_plot(select_var, select_timestep, select_lat, select_lon):
    min_lat, max_lat = select_lat
    min_lon, max_lon = select_lon
    var = select_var
    t = select_timestep

    area = mmap(subpage_map_projection="cylindrical", subpage_lower_left_longitude=int(min_lon),
                subpage_lower_left_latitude=int(min_lat), subpage_upper_right_longitude=int(max_lon), 
                subpage_upper_right_latitude=int(max_lat)) 
 
    ##settings of the coastlines attributes
    coast = mcoast(map_coastline_land_shade = "on", map_coastline_land_shade_colour = "cream", 
                   map_grid_line_style = "dash", map_grid_colour = "brown", map_label_colour = "brown",
                   map_coastline_colour = "brown" ) 
    # CONTOUR
    contour = mcont( contour_automatic_setting = "ecmwf",legend = "on" )

    #Definition of the legend
    legend = mlegend(legend='on', legend_display_type='continuous', 
                     legend_text_colour='charcoal', legend_text_font_size=0.4)  
    title = mtext(text_lines = var, text_font_size = "0.7", text_colour = "charcoal" )

    if var == 'tp':
        aux = gtp
    elif var == 'msl':
        aux = gmsl
    elif var == '2t':
        aux = g2t
        
        
    display(plot( area, coast, aux[t], contour, title, legend))

# RESULT

In [46]:
widgets.interactive(update_plot, 
                    select_var=widget_select_ssvar, select_timestep=widget_sel_timestep,
                    select_lat=latitude_slider, 
                    select_lon=longitude_slider)

interactive(children=(Dropdown(description='Sea surface variable', options=('msl', '2t'), value='msl'), Select…

In [47]:
widgets.interactive(update_plot, 
                    select_var=widget_select_atmvar, select_timestep=widget_sel_timestep,
                    select_lat=latitude_slider, 
                    select_lon=longitude_slider)

interactive(children=(Dropdown(description='Atmospheric variable', options=('tp',), value='tp'), SelectionSlid…